In [14]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
from sklearn.preprocessing import LabelEncoder
from lightgbm import LGBMModel,LGBMRegressor,LGBMClassifier
import functools

In [7]:
def Labelencoder(y):
    labencoder=LabelEncoder()
    #encoder=OneHotEncoder(sparse=False)
    result=labencoder.fit_transform(y)
    #result=encoder.fit_transform(inter.reshape(-1,1))
    result=np.ndarray.tolist(result)
    return result

In [8]:
def missing(x):
    temp=[]
    for i in range(len(x)):
        if pd.isna(x[i]):
            temp.append(True)
        else:
            temp.append(False)
    return(temp)

In [16]:
def fillmissing(z,typ):
    mis=missing(z)
    temp=[]
    for i in range(len(mis)):
        if mis[i]==False:
            temp.append(z[i])
    if typ=='mean':
        mean=functools.reduce(lambda x, y: x + y, temp) / len(temp)
        for i in range(len(mis)):
            if mis[i]==True:
                z[i]=int(mean)
    if typ=='mode':
        mode=max(set(temp), key=temp.count)
        for i in range(len(mis)):
            if mis[i]==True:
                z[i]=mode
    return(z)

In [25]:
test_matrix=pd.read_csv('./test_matrix.csv')

In [26]:
train_matrix=pd.read_csv('./train_matrix.csv')

In [27]:
for i in train_matrix.columns:
    if i !='Unnamed: 0':
        train_matrix[i]=fillmissing(train_matrix[i],'mode')
    else:
        train_matrix[i]=fillmissing(train_matrix[i],'mean')

In [28]:
train_matrix_Y=train_matrix['train_Class']
del train_matrix['train_Class']
del train_matrix['Unnamed: 0']
train_matrix_X=train_matrix

In [29]:
t_data_x=train_matrix_X
t_data_y=train_matrix_Y
p_data_x=test_matrix
loss=1
k=0
temp_loss=[]
X_temp=t_data_x
Y_temp=t_data_y
while loss!=0:
    model=LGBMClassifier(objective="binary",metric='auc',num_iterations = 100,class_weight='balanced')
    model.fit(X=X_temp,y=Y_temp)
    predict=model.predict(X=p_data_x)
    try:
        p_data_y=Y_temp.iloc[1334:2334]
    except:
        print('Expected')
    k+=1
    if k==1:
        X_temp=t_data_x.append(p_data_x,ignore_index=True)
        Y_temp=t_data_y.append(pd.DataFrame(predict),ignore_index=True)
    else:
        loss=0
        for i in range(len(p_data_y)):
            if (p_data_y.loc[i]!=predict[i]):
                loss+=1
        temp_loss.append(loss)
        if(loss!=0):
            Y_temp=t_data_y.append(pd.DataFrame(predict),ignore_index=True)
        else:
            break

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


ValueError: Number of features of the model must match the input. Model n_features_ is 1046 and input n_features is 1047 